In [ ]:
import tensorflow as tf


from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

import os
import tqdm
import numpy as np
import pandas as 
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt

In [ ]:
data_directory_pre = '../Gesture Image Pre-Processed Data'

In [ ]:
img_labels = [i for i in os.listdir(data_directory_pre)]
img_labels.sort()
print(img_labels)


In [ ]:
path0='../Gesture Image Pre-Processed Data/2/1.jpg'
image=cv2.imread(path0)
print(image.shape)
plt.imshow(image)

In [ ]:
def plot_sample(a, b, index):
    plt.figure(figsize = (15,2))
    plt.imshow(a[index])
    plt.xlabel(img_labels[b[index]])

In [ ]:
dataset_img_pre=[]
img_value_pre=[]
# img_labels=
count=0
for name in img_labels:
    path=os.path.join(data_directory_pre,name)
    # print(path)
    for im in os.listdir(path):
      # print(path,im)
        image=cv2.imread(os.path.join(path,im))
        dataset_img_pre+=[image]
        img_value_pre+=[count]
    count=count+1

In [ ]:
plot_sample(dataset_img_pre,img_value_pre,15964)

In [ ]:
img_dataset_pre=np.array(dataset_img_pre)
img_value_pre=np.array(img_value_pre)
# converting to np arrays

In [ ]:
x_train_pre , x_test_pre, y_train_pre, y_test_pre = train_test_split(img_dataset_pre,img_value_pre, test_size=0.2, random_state=42)
# splitting

In [ ]:
plot_sample(x_train_pre,y_train_pre,1256)


In [ ]:
x_train=x_train_pre/255.0
x_test=x_test_pre/255.0

In [ ]:
# data augmentation
dataAugmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip("horizontal", input_shape=(50,50,3)),
])

In [ ]:
# creating the model 
hgrmodel_pre = tf.keras.models.Sequential([
        dataAugmentation,
        tf.keras.layers.Conv2D(32,3, padding='same',activation='relu'),
        tf.keras.layers.MaxPooling2D(),
    
        tf.keras.layers.Conv2D(64,3, padding='same',activation='relu'),
        tf.keras.layers.MaxPooling2D(),

        tf.keras.layers.Dropout(0.2),
    
        tf.keras.layers.Conv2D(64,2, padding='valid',activation='relu'),
        tf.keras.layers.MaxPooling2D(),
    
        # tf.keras.layers.Dropout(0.2),
    
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(37, activation='softmax')
])

hgrmodel_pre.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',        
        metrics=['accuracy'])

In [ ]:
hgrmodel_pre.summary()

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
# creating the model 
# %%timeit -n1 -r1
with tf.device('/GPU:0'):
    hgrmodel_pre.fit(x_train_pre,y_train_pre, epochs=10)

In [ ]:
hgrmodel_pre.evaluate(x_test_pre, y_test_pre)

In [ ]:
hgrmodel_pre.save('hgrmodel_pre.h5')
